# Retention Time Prediction 

This notebook is prepared to be run in Google [Colaboratory](https://colab.research.google.com/). In order to train the model faster, please change the runtime of Colab to use Hardware Accelerator, either GPU or TPU.

This notebook presents a short walkthrough the process of reading a dataset and training a model for retention time prediction. The dataset is an example dataset extracted from a ProteomTools dataset generated in the **Chair of Bioanalytics** at the **School of Life Sciences** at the **Technical University of Munich**.

The framework being used is a custom wrapper on top of Keras/TensorFlow. The working name of the package is for now DLOmix -  `dlomix`.

In [ ]:
# install the DLOmix package in the current environment using pip

!python -m pip install -q dlomix

The available modules in the framework are as follows:

In [ ]:
import numpy as np
import pandas as pd
import dlomix
from dlomix import constants, data, eval, layers, models, pipelines, reports
print([x for x in dir(dlomix) if not x.startswith("_")])

- `constants`: constants to be used in the framework (e.g. Aminoacid alphabet mapping)
- `data`:  classes for representing dataset, wrappers around HuggingFace datasets to process input data and generate tensor datasets
- `eval`: custom evaluation metrics implemented in Keras/TF to work as `metrics` for model training
- `layers`: custom layer implementation required for the different models
- `models`: different model implementations for Retention Time Prediction
- `pipelines`: complete pipelines to run a task (e.g. Retention Time prediction)

**Note**: reports and pipelines are work-in-progress, some funtionalities are not complete.

## 1. Load Data

We can import the dataset class and create an object of type `RetentionTimeDataset`. This object wraps around a Hugging Face dataset that can generate TensorFlow Dataset objects or Torch Dataset for training, validation, or testing. This can be controlled by the arguments `val_ratio`, `val_data_source`, and `test_data_source`.

In [ ]:
from dlomix.data import RetentionTimeDataset

In [ ]:
TRAIN_DATAPATH = 'https://raw.githubusercontent.com/wilhelm-lab/dlomix/develop/example_dataset/proteomTools_train_val.csv'
BATCH_SIZE = 64

rtdata = RetentionTimeDataset(data_source=TRAIN_DATAPATH,
                              sequence_column="sequence",
                              label_column="irt",
                              data_format="csv",
                              max_seq_len=30, batch_size=BATCH_SIZE, val_ratio=0.2)

Now we have an RT dataset that can be used directly with standard or custom `Keras` models. This wrapper contains the splits we chose when creating it. In our case, they are training and validation splits. To get the TF Dataset, we call the attributes `.tensor_rain_data` and `.tensor_val_data`.

In [ ]:
"Hugging Face Dataset", rtdata

In [ ]:
 "Training examples", len(rtdata["train"])

In [ ]:
 "Training examples", len(rtdata["val"])

## 2. Model

We can now create the model. We will use a simple Prediction with a conv1D encoder. It has the default working arguments, but most of the parameters can be customized.

**Note**: Important is to ensure that the padding length used for the dataset object is equal to the sequence length passed to the model.

In [ ]:
from dlomix.models import RetentionTimePredictor

In [ ]:
model = RetentionTimePredictor(seq_length=30)

## 3. Training

We can then train the model like a standard Keras model. The optimizer parameters here are from Prosit, but other optimizer parameters can be used.  

In [ ]:
#imports

from dlomix.eval import TimeDeltaMetric

In [ ]:
# compile the model  with the optimizer and the metrics we want to use, we can add our custom timedelta metric

# you can also import tensorflow and build your custom optimizer object and pass it

model.compile(optimizer='adam', 
              loss='mse',
              metrics=[TimeDeltaMetric()])

We store the result of training so that we can explore the metrics and the losses later. We can pass the attributes `tensor_train_data` and `tensor_val_data`, where each returns the corresponding TensorFlow Dataset object.

In [ ]:
history = model.fit(rtdata.tensor_train_data,
                    validation_data=rtdata.tensor_val_data,
                    epochs=20)

## 3. Testing and Reporting

We can create a test dataset to test our model. Additionally, we can use the reporting module to produce plots and evaluate the model.

Note: the reporting module is still in progress and some functionalities might easily break.

In [ ]:
# create the dataset object for test data

TEST_DATAPATH = 'https://raw.githubusercontent.com/wilhelm-lab/dlomix/develop/example_dataset/proteomTools_test.csv'

test_rtdata = RetentionTimeDataset(test_data_source=TEST_DATAPATH,
                                   data_format="csv",
                                   sequence_column="sequence",
                                   label_column="irt",
                                   max_seq_len=30, batch_size=32)

In [ ]:
# use model.predict from keras directly on the testdata

predictions = model.predict(test_rtdata.tensor_test_data)

# we use ravel from numpy to flatten the array (since it comes out as an array of arrays)
predictions = predictions.ravel()

In [ ]:
# we can get the targets of a specific split to calcualte evaluation metrics against predictions
# we access the underlying hugging face split named "test" in this case and access the label column "irt"

test_targets = test_rtdata["test"]["irt"]

In [ ]:
test_targets[:5], predictions[:5]

In [ ]:
from dlomix.reports import RetentionTimeReport

In [ ]:
# create a report object by passing the history object and plot different metrics
report = RetentionTimeReport(output_path="./output", history=history)

In [ ]:
report.plot_keras_metric("loss")

In [ ]:
report.plot_keras_metric("mean_absolute_error")

In [ ]:
report.plot_keras_metric("timedelta")

In [ ]:
# calculate R2  given the targets and the predictions of the test data
report.calculate_r2(test_targets, predictions)

In [ ]:
report.plot_density(test_targets, predictions)

In [ ]:
report.plot_residuals(test_targets, predictions, xrange=(-30, 30))

## 4. Saving and Loading Models

Models can be saved normally the same Keras models would be saved. It is better to save the weights and the not the model since it makes it easier and more platform-indepdent when loading the model again. The extra step needed is to create a model object and then load the weights.

In [ ]:
# save the model weights

save_path = "./output/rtmodel"
model.save_weights(save_path)

In [ ]:
# models can be later loaded by creating a model object and then loading the weights

trained_model = RetentionTimePredictor(seq_length=30)
trained_model.load_weights(save_path)

We can compare the predictions to make sure that the model was loaded correctly.

In [ ]:
new_predictions = trained_model.predict(test_rtdata.tensor_test_data)

In [ ]:
new_predictions = new_predictions.ravel()

In [ ]:
import numpy as np
# confirm all old and new predictions are the same
np.allclose(predictions, new_predictions)

In [ ]:
import pandas as pd
results_df = pd.DataFrame({"sequence": test_rtdata["test"]["sequence"],
                           "irt": test_rtdata["test"]["irt"],
                           "predicted_irt": predictions})

results_df.to_csv("./output/predictions_irt.csv", index=False)

In [ ]:
pd.read_csv("./output/predictions_irt.csv")